# Kryptools

Warning: These tools are intended for experimenting with cryptographic algorithms. They are not intended for use in real applications.

## Zmod: Ring of integers modulo n

To define $\mathbb{Z}_5$, the [ring of integers modulo the prime $5$](https://en.wikipedia.org/wiki/Multiplicative_group_of_integers_modulo_n) (in this case a [Galois field](https://en.wikipedia.org/wiki/Finite_field)), use

In [1]:
from kryptools import Zmod

Z_5 = Zmod(5)

To declare $3$ as an element of $\mathbb{Z}_5$ use

In [2]:
Z_5(3)

3

Basic arithmetic operations work as expected:

In [3]:
Z_5(3) + Z_5(2)

0

In [4]:
Z_5(3) ** -1

2

If you add or multiply integers to an element of $\mathbb{Z}_5$, they will be automatically be converted:

In [5]:
2 * Z_5(3) + 1

2

You can also compute square roots (if the number is not a quadratic residue, a ValueError is raised):

In [6]:
Z_5(4).sqrt()

3

A list of random points can be obtained with (if no argument is given, a single point is retruned):

In [7]:
Z_5.random(4)

[3, 3, 0, 2]

To determine the multiplicative [order](https://en.wikipedia.org/wiki/Order_(group_theory)) of an element in the group $\mathbb{Z}_n^*$ use:

In [8]:
Z_5(3).order()

4

You can also check if an element is a [generator](https://en.wikipedia.org/wiki/Primitive_root_modulo_n) (a.k.a primitive root) of $\mathbb{Z}_n^*$:

In [9]:
Z_5(3).is_generator()

True

You can get one generator or all generators

In [10]:
Z_5.generator(), list(Z_5.generators())

(2, [2, 3])

as well as all elements of $\mathbb{Z}_n^*$

In [11]:
list(Z_5.star())

[1, 2, 3, 4]

To test if $\mathbb{Z}_n^*$ is cyclic use

In [12]:
Z_5.is_cyclic()

True

For [Learning With Errors](https://en.wikipedia.org/wiki/Learning_with_errors) a symmetric representative is frequently required:

In [13]:
[a.sharp() for a in Z_5]

[0, 1, 2, -2, -1]

The absolute value will be the absolute value of the symmetric representative:

In [14]:
[abs(a) for a in Z_5]

[0, 1, 2, 2, 1]

If you set the option `short` to `False`, then $\pmod{5}$ will be added:

In [15]:
Z_5.short = False
Z_5(6)

1 (mod 5)

## Galois field $\mathrm{GF}(2^n)$

There is also some support for playing with the [Galois field](https://en.wikipedia.org/wiki/Finite_field) $\mathrm{GF}(2^n)$. Elements are specified by integers from $0$ to $2^n-1$, which are identified with polynomials $\mathbb{Z}_2[x]_{m(x)}$ whose coefficients are the binary digits of the integer. Multiplication is polynomial multiplication modulo a given irreducible polynomial $m(x)$ (the modulus).
If the modulus has the $n$'th bit set, the lowest bit corresponds to the constant coefficient (like in AES). Otherwise, the bit order is reversed (like for Ghash).
The default polynomial is a [Conway polynomial](https://en.wikipedia.org/wiki/Conway_polynomial_(finite_fields)) from [Frank Lübeck’s list](https://www.math.rwth-aachen.de/~Frank.Luebeck/data/ConwayPol/index.html).

To define the Galois field GF(2^8) for [AES](https://en.wikipedia.org/wiki/Rijndael_MixColumns) with polynomial $1 + x + x^3 + x^4 + x^8$ use

In [16]:
from kryptools import GF2

gf = GF2(8, modulus=0b100011011)  # x^8 + x^4 + x^3 + x + 1 = AES

Note: If you do not specify the modulus, the corresponding Conway polynomial $1 + x^2 + x^3 + x^4 + x^8$ will be used.

To declare $3$ as an element of our Galois field use (elements are displayed as hex numbers)

In [17]:
gf(3)

03

The usual arithmetic operations are supported.

In [18]:
gf(2).sqrt()**2 == gf(2)

True

In [19]:
gf(2) * gf(135)

15

Note that if you multiply with integers, the multiplication will be modulo the characteristic 2:

In [20]:
2 * gf(2), gf(2) * gf(2)

(00, 04)

Addition between integers and elements of $GF(2^n)$ is not supported and will raise an error.

Elements from $\mathrm{GF}(2^n)$ can be convertet to `int`, `bytes`, or to a polynomial (see the `Poly` class below):

In [21]:
gf(2).poly()

x

The irreducible polynomial used to define the field can be obtained with

In [22]:
gf.poly()

x^8 + x^4 + x^3 + x + 1

The minimal polynomial of some element is the smallest (degree) polynomial which vanishes at this element and has coefficients in $GF(2)$:

In [23]:
gf(2).minpoly()

x^8 + x^4 + x^3 + x + 01

To compute the multiplicative order use:

In [24]:
assert gf(3) ** gf(3).order() == gf(1)
gf(3).order()

255

To get a generator (primitive root) of the multiplicative group use:

In [25]:
gf.generator()

03

For convenience, the AES Galois field is also available as `GF2_aes`. It has an additional method for applying the AES sbox (to get the inverse sbox, use the option `inv=True`):

In [26]:
from kryptools import GF2_aes

GF2_aes(0).sbox()

63

Note that for fields with not too many elements you can use the option `cached = True` when creating the field or `gf.cache()` later on to create an exp/log table with respect to the geneartor `gf.generator()` to speed up the multiplication.

### Showcase: Ghash

For [Ghash](https://en.wikipedia.org/wiki/Galois/Counter_Mode) the polynomial is $1 + x + x^2 + x^7 + x^{128}$. You can explicitely define the corresponding Galois field using

In [27]:
gf = GF2(128, modulus=0b11100001 << 120)  # 1 + x + x^2 + x^7 (+ x^128) = Ghash

For convenience, it is also avaiable as `GF2_ghash`:

In [28]:
from kryptools import GF2_ghash


def ghash(data: bytes, h: bytes) -> bytes:
    "Compute ghash of a bytestring `data` with a given key `h`."
    # First pad data such that it is a multiple of the block length 16*8=128
    pad = len(data) % 16
    if pad > 0:
        data += b"\x00" * (16 - pad)
    # Now compute ghash recursively
    h = GF2_ghash(h)
    g = GF2_ghash(0)
    for i in range(len(data) // 16):
        g = h * (g + GF2_ghash(data[i * 16 : (i + 1) * 16]))
    return bytes(g)


h = b"7\xa0\x8e\xd3\x00\x93\xa13\xb1\xbbJ\xe0\xb8\xf3`\x1f"  # 128 bit key
text = b"A boring message"

ghash(text, h)

b'\xcc\x16\x99\x1d\x8d`\xadu\xc9+\xe1,\x98\x13j\x17'

## Primes

A tuple of [primes](https://en.wikipedia.org/wiki/Prime_number) below a bound $B$ can be obtained using the [sieve of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes):

In [29]:
from kryptools import sieve_eratosthenes

sieve_eratosthenes(20)

(2, 3, 5, 7, 11, 13, 17, 19)

The Prime-counting function $\pi(x)$ and the primorial $\#x$ is implemented by counting the number of primes returned by the sieve of Eratosthenes:

In [30]:
from kryptools import prime_pi, primorial

prime_pi(20), primorial(7)

(8, 210)

To test if a number is probable prime you can use

In [31]:
from kryptools import is_prime

is_prime(primorial(13) + 1)

False

For numbers below $3317044064679887385961981$ it performs a deterministic variant of the [Miller-Rabin test](https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test)

In [32]:
from kryptools import miller_rabin_test

miller_rabin_test(5459, [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41])

False

Otherwise it combines trial division with small primes (for speed), a Miller-Rabin test with base $2$, and a [strong Lucas test](https://en.wikipedia.org/wiki/Lucas_pseudoprime#Implementing_a_Lucas_probable_prime_test) ([Baillie–PSW primality test](https://en.wikipedia.org/wiki/Baillie%E2%80%93PSW_primality_test)).

In [33]:
from kryptools import lucas_test

lucas_test(5459)

True

The next/previous prime larger/smaller or equal to a given number

In [34]:
from kryptools import next_prime, previous_prime

next_prime(100), previous_prime(100)

(101, 97)

A pseudorandom prime (based on `secure.randbits`, so hopefully cryptographically safe) of given bit length

In [35]:
from kryptools import random_prime

p = random_prime(128)
assert is_prime(p)
assert p.bit_length() == 128

Similarly, a pseudorandom [safe prime](https://en.wikipedia.org/wiki/Safe_and_Sophie_Germain_primes) $p$ of given bit length with $2$ generating the large subgroup, $\text{ord}(2)=(p-1)/2$:

In [36]:
from kryptools import is_safeprime, order, random_safeprime

p = random_safeprime(128)
assert is_safeprime(p)
assert p.bit_length() == 128
assert order(2, p) == (p - 1) // 2

A list of the first safe primes together with the corresponding Sophie Germain primes can be optained as follows:

In [37]:
[((p - 1) // 2, p) for p in sieve_eratosthenes(200) if is_safeprime(p)]

[(2, 5),
 (3, 7),
 (5, 11),
 (11, 23),
 (23, 47),
 (29, 59),
 (41, 83),
 (53, 107),
 (83, 167),
 (89, 179)]

Finally, a pseudorandom [strong prime](https://en.wikipedia.org/wiki/Strong_prime) $p$ with factors of $p\pm 1$ of given bit length using Gordon's algorithm:

In [38]:
from kryptools import random_strongprime

t, s, r, p = random_strongprime(128)
assert is_prime(p) and is_prime(t) and is_prime(s) and is_prime(r)
assert (p + 1) % s == 0 and (p - 1) % r == 0 and (r - 1) % t == 0
p.bit_length()

264

## Number theory

To perform the [extended Euclidean algorithm](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm) use

In [39]:
from kryptools.nt import egcd

a, b = 7, 8
g, x, y = egcd(a, b)

assert x * a + y * b == g
g, x, y

(1, -1, 1)

To solve a list of congruences using the [Chinese Remainder Theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem) use

In [40]:
from kryptools import crt

a, m = [1, 2, 3], [2, 3, 5]
sol = crt(a, m)

assert [sol % p for p in m] == a
sol

23

The moduli are assumed to be coprime. If not, you can use:

In [41]:
m = [4, 8, 12, 30]
x = 45
a = [x % mi for mi in m]

assert crt(a, m, coprime=False) == x

To compute the [continued fraction](https://en.wikipedia.org/wiki/Continued_fraction) expansion of a rational number use

In [42]:
from fractions import Fraction

Fraction.__repr__ = Fraction.__str__
from kryptools import cf, convergents

cf(Fraction(18, 23))

[0, 1, 3, 1, 1, 2]

To get the convergents use

In [43]:
convergents(_)

[0, 1, 3/4, 4/5, 7/9, 18/23]

To compute the [Jacobi symbol](https://en.wikipedia.org/wiki/Jacobi_symbol) use

In [44]:
from kryptools import jacobi_symbol

jacobi_symbol(4, 7)

1

Since it equals $+1$, the number is a [quadratic residue](https://en.wikipedia.org/wiki/Quadratic_residue) and has a square root modulo the prime $7$

In [45]:
from kryptools import sqrt_mod

sqrt_mod(4, 7)

2

To compute [Euler's $\varphi$ function](https://en.wikipedia.org/wiki/Euler%27s_totient_function) (a.k.a totient function) or [Carmichael's $\lambda$ function](https://en.wikipedia.org/wiki/Carmichael_function) (a.k.a reduced totient function) or [Möbius' $\mu$ function](https://en.wikipedia.org/wiki/Moebius_function) use

In [46]:
from kryptools import carmichael_lambda, euler_phi, moebius_mu

n = 100
euler_phi(n), carmichael_lambda(n), moebius_mu(n)

(40, 20, 0)

A test if a number is a [Carmichael number](https://en.wikipedia.org/wiki/Carmichael_number):

In [47]:
from kryptools import is_carmichael_number

is_carmichael_number(561)

True

To compute the order of an element of the [multiplicative group of integers modulo $n$](https://en.wikipedia.org/wiki/Multiplicative_group_of_integers_modulo_n), $\mathbb{Z}_n^*$, use

In [48]:
from kryptools.nt import order

order(5, 7)

6

### Showcase: Asmuth-Bloom scheme

Using the Chinise Remainder Theorem you can share a secret $s\in\mathbb{Z}_m$ among $n$ persons, such that $r$ persons can recover the secret, using the [Asmuth-Bloom scheme](https://en.wikipedia.org/wiki/Secret_sharing_using_the_Chinese_remainder_theorem#Asmuth%E2%80%93Bloom_threshold_secret_sharing_scheme). First we need to generate suitable parameters. We choose primes for the moduli to ensure that they are coprime.

In [49]:
from math import prod

from kryptools import crt, next_prime

n, r = 5, 3
m = 2**32  # max size of the secret

M = []
mm = 2 * m
for _ in range(n):
    mm = next_prime(mm + 1)
    M.append(mm)

M1 = prod(M[:r])
assert m * prod(M[1 - r :]) < M1  # Asmuth-Bloom condition
M

[8589934609, 8589934621, 8589934627, 8589934631, 8589934651]

Now we can split a secret $s$ into $n$ pieces:

In [50]:
from random import randint, seed

seed(0)

s = randint(0, m - 1)
t = randint(1, (M1 - s) // m)
x = s + t * m
del t  # t is no longer needed and should be deleted for security reasons
assert x < M1

A = [x % mm for mm in M]
A

[953188841, 2267150034, 7376225206, 341660855, 3090211041]

Let us check that it worked:

In [51]:
from random import sample

smpl = sample(range(n), r)

assert crt([A[i] for i in smpl], [M[i] for i in smpl]) % m == s

## Factoring integers

To factor an integer $n$ into its prime factors use

In [52]:
from kryptools import factorint

n = 2**128 + 1
factorint(n, verbose=1)

Factoring: 340282366920938463463374607431768211457 (39 digits)
Trial division found: []
Round 1 (B1=5000)
Factors found (ecm):  [59649589127497217, 5704689200685129054721]


{59649589127497217: 1, 5704689200685129054721: 1}

There are also tests if a number is a [perfect power](https://en.wikipedia.org/wiki/Perfect_power) (if it is not, `None` will be returned)

In [53]:
from kryptools import perfect_power

perfect_power(100)

(10, 2)

or a prime power (if it is not, `None` will be returned)

In [54]:
from kryptools import prime_power

prime_power(9973**100)

(9973, 100)

For factoring you can also use a specific algorithm (if no factor is found, `None` is returned):

[Fermat's method](https://en.wikipedia.org/wiki/Fermat%27s_factorization_method)

In [55]:
from kryptools.factor_fmt import factor_fermat

factor_fermat(832283)

[487, 1709]

[Pollard $p-1$](https://en.wikipedia.org/wiki/Pollard%27s_p_%E2%88%92_1_algorithm)

In [56]:
from kryptools.factor_pm1 import factor_pm1

factor_pm1(832283)

1709

[Pollard $\rho$](https://en.wikipedia.org/wiki/Pollard%27s_rho_algorithm)

In [57]:
from kryptools.factor_rho import factor_rho

factor_rho(832283)

487

[Lentstra's ECM](https://en.wikipedia.org/wiki/Lenstra_elliptic-curve_factorization)

In [58]:
from kryptools.factor_ecm import factor_ecm

factor_ecm(832283)

487

A variant of [Dixon's method](https://en.wikipedia.org/wiki/Dixon%27s_factorization_method)

In [59]:
from kryptools.factor_dix import factor_dixon

factor_dixon(832283)

1709

[Quadratic sieve](https://en.wikipedia.org/wiki/Quadratic_sieve) (with a single polynomial)

In [60]:
from kryptools.factor_qs import factor_qs

factor_qs(832283)

1709

## Discrete logarithms

To solve a [discrete log](https://en.wikipedia.org/wiki/Discrete_logarithm) problem use

In [61]:
from random import randint

from kryptools import dlog

p, m, a = [557639, 278819, 2]  # m is the order of a in Z_p
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog(a, b, p) == x

You can also use a specific algorithm:

Exhaustive search:

In [62]:
from kryptools.dlp import dlog_naive

p, m, a = [557639, 278819, 2]
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog(a, b, p, m) == x

[Pollard $\rho$](https://en.wikipedia.org/wiki/Pollard%27s_rho_algorithm) (with Brent's or Floyd's [cycle detection algorithm](https://en.wikipedia.org/wiki/Cycle_detection))

In [63]:
from kryptools.dlp_rho import dlog_rho

p, m, a = [557639, 278819, 2]
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog_rho(a, b, p, m, brent=True) == x

Shanks's [baby step/giant step](https://en.wikipedia.org/wiki/Baby-step_giant-step) algorithm

In [64]:
from kryptools.dlp_bsgs import dlog_bsgs

p, m, a = [557639, 278819, 2]
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog_bsgs(a, b, p, m) == x

[Index calculus](https://en.wikipedia.org/wiki/Index_calculus_algorithm)

In [65]:
from kryptools.dlp_ic import dlog_ic

p, m, a = [24570203447, 12285101723, 2]
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog_ic(a, b, p, m, verbose=1) == x

Factorbase: bound = 359, size = 72, max_trys = 3000
Success after 69 relations out of 73.


Quadratic sieve

In [66]:
from kryptools.dlp_qs import dlog_qs

p, m, a = [28031135240181527, 14015567620090763, 2]
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog_qs(a, b, p, m, verbose=1) == x

Factorbase: bound = 1903,  size = 291 + 470 = 761, max_trys = 32530
Done sieving.
Success after 476 relations out of 762.


## Linear Algebra

The basic class for working with [matrices](https://en.wikipedia.org/wiki/Matrix_(mathematics)) is `Matrix`, where a matrix is given as list of rows:

In [67]:
from kryptools import Matrix

M = Matrix([[1, 2, 3], [3, 4, 5]])
N = Matrix([[1], [3], [4]])
M * N

[ 19 ]
[ 35 ]

Note that `Matrix([[1, 2, 3]])` is a $1\times 3$ matrix, that is, a row vector, while `Matrix([[1], [2], [3]]) == Matrix([1, 2, 3])` is a $3\times 1$ matrix, that is, a column vector.

You can customize the ASCII display of a matrix using:

In [68]:
Matrix.print_pre, Matrix.print_sep, Matrix.print_post = "(", " ", ")"

print(M)

(1 2 3)
(3 4 5)


To reset the default use

In [69]:
Matrix.print_pre, Matrix.print_sep, Matrix.print_post = "[ ", ", ", " ]"

print(M)

[ 1, 2, 3 ]
[ 3, 4, 5 ]


Slicing to access (e.g.) rows:

In [70]:
M[:, 0]

[ 1 ]
[ 3 ]

Also works with assignments, e.g., to swap two columns:

In [71]:
M[0, :], M[1, :] = M[1, :], M[0, :]
M

[ 3, 4, 5 ]
[ 1, 2, 3 ]

To delete a column:

In [72]:
del M[-1, :]
M

[ 3, 4, 5 ]

The coefficients can be from a ring or field, e.g. `Fraction`. This can be done either by using `ring=Fraction` during creation or using the `map` method which applies a given function to all elements. Integer matrices are automatically converted to `Fraction` to avoid numerical problems. In particular, this package is **not intended for numerical computations**!

In [73]:
M = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 12]])

We can compute the [row reduced echelon form](https://en.wikipedia.org/wiki/Row_echelon_form)

In [74]:
M.rref()

[ 1, 0, 0 ]
[ 0, 1, 0 ]
[ 0, 0, 1 ]

the [determinant](https://en.wikipedia.org/wiki/Determinant)

In [75]:
M.det()

-9

the [rank](https://en.wikipedia.org/wiki/Rank_(linear_algebra)) and [nullity](https://en.wikipedia.org/wiki/Nullity_(linear_algebra))

In [76]:
M.rank(), M.nullity()

(3, 0)

a basis for the [kernel](https://en.wikipedia.org/wiki/Kernel_(linear_algebra))

In [77]:
M.kernel()

[ 0 ]
[ 0 ]
[ 0 ]

or the [inverse](https://en.wikipedia.org/wiki/Invertible_matrix)

In [78]:
Mi = M.inv()
assert M * Mi == M.eye()
Mi

[ -4/3,    0,  1/3 ]
[  2/3,    1, -2/3 ]
[  1/3, -2/3,  1/3 ]

Solving a linear system $A x = b$ (it will return one solution or `None` if no solution exists):

In [79]:
A = Matrix([[2, 1], [2, 1]], ring = Fraction)
b = Matrix([1, 1], ring = Fraction)

sol = A.solve(b)
if sol:
    assert A * sol == b
sol

[ 1/2 ]
[   0 ]

Same works if the coefficients are from a finite field or ring:

In [80]:
from kryptools import Zmod

gf = Zmod(11)

M = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 12]], ring=gf)
M.det()

2

In [81]:
Mi = M.inv()
assert M * Mi == M.eye()
Mi

[ 6, 0, 4 ]
[ 8, 1, 3 ]
[ 4, 3, 4 ]

Note that `eye` will give you the identity matrix of the same dimensions. There is also a function `zeros` to create a zero matrix. Moreover, both are also available as stand alone functions and take the desired dimensions as arguments.

In [82]:
from kryptools import eye, zeros

zeros(3,2)

[ 0, 0 ]
[ 0, 0 ]
[ 0, 0 ]

There is also limited support if $\mathbb{Z}_n$ is a ring (and no field):

In [83]:
from kryptools import Zmod, Matrix

ring = Zmod(10)

M = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 12]], ring=ring)
M.rref()

[ 1, 0, 0 ]
[ 0, 1, 0 ]
[ 0, 0, 1 ]

For example you can compute the inverse

In [84]:
M.inv()

[ 2, 0, 7 ]
[ 4, 1, 6 ]
[ 7, 6, 7 ]

or solve linear equations

In [85]:
b = ring([1,2,2])
x  = M.solve(b)
if x is not None:
    assert list(M * x) == b
x

[ 6 ]
[ 8 ]
[ 3 ]

There is also support for computing the [Hermite normal form](https://en.wikipedia.org/wiki/Hermite_normal_form) normal form

In [86]:
M.map(int)
M.hnf()

[ 21, 16, 6 ]
[  0,  1, 0 ]
[  0,  0, 1 ]

and the [Smith normal form](https://en.wikipedia.org/wiki/Smith_normal_form)

In [87]:
D, S, T = M.snf()
assert D == S * M * T
D

[ 1, 0,  0 ]
[ 0, 1,  0 ]
[ 0, 0, 21 ]

For binary matrices (with coefficients in $\mathbb{Z}_2$ there is a special class `BinaryMatrix` which stores the rows as integers whose bits (counting starrts from the left) correspond to the entries. It supports most of the functionality of the `Matrix` class will allowing for much faster rwo operations.

In [88]:
from kryptools import BinaryMatrix

M = BinaryMatrix([[0, 1], [1, 0]])
M

[01]
[10]

To apply a vector to the matrix (which can be given as a list of bits or as an integer) you can use:

In [89]:
M.apply([1, 0]), M.apply(2)

([0, 1], 1)

To convert between bits and integers you can use:

In [90]:
M.from_bits([1,0]), M.to_bits(2)

(2, [1, 0])

To get an ordinary `Matrix` you can use

In [91]:
M = Matrix(M.bitmatrix(), ring = Zmod(2))
M

[ 0, 1 ]
[ 1, 0 ]

and to get back you can use

In [92]:
BinaryMatrix(M.matrix)

[01]
[10]

## Lattice

To compute the [Gram-Schmidt decomposition](https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process) use

In [93]:
from fractions import Fraction
from kryptools import Matrix, gram_schmidt

V = Matrix([[5, 8], [0, 1]], ring=Fraction)
Vs, M = gram_schmidt(V)
assert V == Vs * M
Vs

[ 5, 0 ]
[ 0, 1 ]

To compute the [lattice](https://en.wikipedia.org/wiki/Lattice_(group)) determinant/[gram determinant](https://en.wikipedia.org/wiki/Gram_matrix) use

In [94]:
from kryptools import gram_det

gram_det(V)

5.0

To compute the Hadamard ratio use

In [95]:
from kryptools import hadamard_ratio

hadamard_ratio(V)

0.3521856535823236

Approximately solving the closest vector problem using Babai's rounding algorithm

In [96]:
from kryptools import babai_round_cvp

V = Matrix([[5, 8], [0, 1]])
x = Matrix([5.2, 0.4])

a = babai_round_cvp(x, V)
print((a - x).norm())
a

5.215361924162119


[ 0 ]
[ 0 ]

Using Lagrange-Gauß reduction gives a better solution

In [97]:
def lagrange_lr(V: Matrix) -> Matrix:
    "Lagrange lattice reduction."
    if V.cols != 2:
        raise ValueError("Lagrange lattice reduction requires dimension two.")
    v1, v2 = V[:, 0], V[:, 1]
    if v1.norm2() > v2.norm2():
        v1, v2 = v2, v1
    v3 = v2 - round(v1.dot(v2) / v1.norm2()) * v1
    while v3.norm2() < v1.norm2():
        v2, v1 = v1, v3
        v3 = v2 - round(v1.dot(v2) / v1.norm2()) * v1
    return Matrix([list(v1), list(v3)]).transpose()

U = lagrange_lr(V)
a = babai_round_cvp(x, U)
print((a - x).norm())
a

0.44721359549995804


[ 5 ]
[ 0 ]

Alternatively we can use Babai's cosest plane algorithm

In [98]:
from kryptools import babai_plane_cvp

V = Matrix([[5, 8], [0, 1]])

a = babai_plane_cvp(x, V)
print((a - x).norm())
a

0.44721359549995804


[ 5 ]
[ 0 ]

Moreover, there is a class form manipulating lattices. The lattice is specified by an initial basis. This basis can then be transformed using furhter operations. For example, you can perfom an
[LLL reduction](https://en.wikipedia.org/wiki/Lenstra%E2%80%93Lenstra%E2%80%93Lov%C3%A1sz_lattice_basis_reduction_algorithm)

In [99]:
from kryptools import Lattice

lat = Lattice( Matrix([[2, 3, -9, -2, 6], [5, 2, -1, 5, 1], [8, -4, 6, -6, -1], [-6, -7, 9, -2, 7], [9, -6, -1, -7, -8]]) )
lat.lll(delta = 1, deep = True, verbose = True)
lat

Starting LLL at index 1
Swaping basis vectors 1 <-> 0
Reduced basis vector 2
Deep insertion of basis vector 2 at 0
Reduced basis vector 2
Reduced basis vector 3
Deep insertion of basis vector 3 at 0
Reduced basis vector 1
Reduced basis vector 3
Reduced basis vector 4
Swaping basis vectors 4 <-> 3
Reduced basis vector 4
Total number of steps: 12


[ -5, -1,  3, 3, -6 ]
[  3, -2,  2, 9, -1 ]
[ -2,  4, -4, 5,  1 ]
[  5, -3, -7, 6, -7 ]
[ -1, -6, -6, 0,  8 ]

The implementation of LLL will handle linearly dependent basis vactors and result in a basis for the lattice provided $\delta> 1/2$. It also performs deep insertions if requested. You could also first compute the Hermite normal form `lat.hnf()` to get rid of linearly dependent vectors or you can try Hermite reduction `lat.hermite()` but the latter might not remove all linearly dependent vectors.

You can also compute a [Hermite-Korkine-Zolotarev](https://en.wikipedia.org/wiki/Korkine%E2%80%93Zolotarev_lattice_basis_reduction_algorithm) reduced basis:

In [100]:
lat.hkz()
lat

[ -5, -1,  3, 3, -6 ]
[  3, -2,  2, 9, -1 ]
[ -2,  4, -4, 5,  1 ]
[  5, -3, -7, 6, -7 ]
[ -1, -6, -6, 0,  8 ]

You can solve the SVP

In [101]:
lat.svp()

[ -5 ]
[  3 ]
[ -2 ]
[  5 ]
[ -1 ]

There are three basic methods available: `hermite`, `lll` and `enum`. The first will apply Hermite reduction before returning the shortest basis vector, the second will apply LLL reduction before returning the shortest basis vector, and the last findes the exact solution via a branch and bound strategy (naturally only feasible for small lattices).

Similarly, you can solve the CVP

In [102]:
b = Matrix([1, 2, 3, 4, 5])
lat.cvp(b)

[ -3 ]
[ -2 ]
[  4 ]
[  7 ]
[  6 ]

There are four basic methods available: `babai_round`, `babai_plane`, `kannan` and `enum`. They will perform both Hermite and LLL reduction and then the requestes method. The last one again findes the exact solution via a branch and bound strategy (again only feasible for small lattices).

To get a pseudorandom [unimodular matrix](https://en.wikipedia.org/wiki/Unimodular_matrix) use

In [103]:
from kryptools import random_unimodular_matrix

random_unimodular_matrix(3, max_val=9)

[ 1, -8, -9 ]
[ 0,  3,  4 ]
[ 4,  2,  9 ]

### Showcase: GGH cryptosystem

For example, to generate a key pair for the [GGH cryptosystem](https://en.wikipedia.org/wiki/GGH_encryption_scheme) we start with a nearly orthogonal matrix

In [104]:
from random import randint, seed

from kryptools import eye

seed(0)

n = 4
M = 100
U = 2 * M * n * eye(n) + Matrix([[randint(-M, M) for i in range(n)] for j in range(n)])
U

[ 798,  94,   7, -90 ]
[ -34, 830,  24,   3 ]
[ 100, -23, 822,  -9 ]
[  49, -45,  29, 735 ]

which has a good Hadamrad ratio

In [105]:
hadamard_ratio(U)

0.9962449639096419

Babai's closest pane algorithm will be able to solve the CVP provided the (sup norm of the) error is smaller than:

In [106]:
from math import floor, inf

from kryptools import babai_plane_bnd

babai_plane_bnd(U, inf)

367.329164515002

To obtain the public key we muliply $U$ with a pseudorandom unimodular matrix:

In [107]:
V = U * random_unimodular_matrix(n)
V

[ -1100279,  2579493, -2669106,  2347653 ]
[  -523004,  1175996, -1260068,  1070213 ]
[ -1524616,  3537925, -3692275,  3219934 ]
[   823300, -1867613,  1986482, -1699706 ]

which results in a small Hadamard ratio

In [108]:
hadamard_ratio(V)

0.0002031966189726793

In this case Babai's closest pane algorithm will be able to solve the CVP provided the (sup norm of the) error is smaller than:

In [109]:
babai_plane_bnd(V, inf)

0.015520618275460585

Now we can regarad the message `FAD!` as a vector of the corresponding ASCII codes $x$ and encrypt the corresponding vector by perturbing the associated lattice vector with a random error $r$:

In [110]:
from math import floor

r_max = floor(babai_plane_bnd(U, inf))
msg = b"FAD!"
assert len(msg) == n

x = Matrix([x for x in msg])
r = Matrix([randint(-r_max, r_max) for _ in range(n)])
y = V * x + r
y

[ -13379202 ]
[ -10538159 ]
[ -21574537 ]
[  15226425 ]

Using the private basis $U$ we can decrypt (we change from integers to `Fraction` for the matrix inversion to avoid rounding errors)

In [111]:
V.map(Fraction)
V.inv() * babai_round_cvp(y, U) == x

True

Using the public basis $V$ decryption fails

In [112]:
V.map(Fraction)
V.inv() * babai_round_cvp(y, V) == x

False

Since our parameters are too small, the system can be broken using LLL reduction:

In [113]:
from kryptools import lll

lat = Lattice(V)
lat.lll()
assert V.inv() * lat.cvp(y) == x

In [114]:
x

[ 70 ]
[ 65 ]
[ 68 ]
[ 33 ]

### Showcase: SIS and ISIS

The [Short Integer Solution problem](https://en.wikipedia.org/wiki/Short_integer_solution_problem) asks for a short solution of the homogenous matrix equation $A x =0$ in $\mathbb{Z}_p$:

In [115]:
from kryptools import Zmod, Matrix, eye
from random import seed
seed(0)

p = 17
gf = Zmod(p)

A = Matrix([[gf.random() for _ in range(5)] for _ in range(3)])
A

[ 12, 13, 1,  8, 16 ]
[ 15, 12, 9, 15, 11 ]
[  6, 16, 4,  9,  4 ]

A basis for the kernel of $A$ is given by

In [116]:
K = A.kernel()
K

[ 12, 15 ]
[  0, 12 ]
[  1,  5 ]
[  1,  0 ]
[  0,  1 ]

Of course in this case we can search for a short vector using an exhaustive search. If the dimensions get too large you can use lattice reduction to do the job. Hence we convert to integers and add $p$ times the identity matrix to our lattice such that vectors get reduced modulo $p$ if this makes them shorter (i.e. we form the corresponding $p$-ary lattice).

In [117]:
L = p * eye(K.rows)
L.append_column(K, ring = int)
L

[ 17,  0,  0,  0,  0, 12, 15 ]
[  0, 17,  0,  0,  0,  0, 12 ]
[  0,  0, 17,  0,  0,  1,  5 ]
[  0,  0,  0, 17,  0,  1,  0 ]
[  0,  0,  0,  0, 17,  0,  1 ]

Now we compute the Hermite normal form to get rid of linearly dependent vectors

In [118]:
L = L.hnf()
L

[ 17,  0,  0, 12, 15 ]
[  0, 17,  0,  0, 12 ]
[  0,  0, 17,  1,  5 ]
[  0,  0,  0,  1,  0 ]
[  0,  0,  0,  0,  1 ]

 and apply LLL:

In [119]:
from kryptools import lll

lll(L)

[ 1, 2, -5,  2, -2 ]
[ 2, 0,  0, -3,  5 ]
[ 0, 3,  1,  1,  6 ]
[ 2, 3,  1, -2, -6 ]
[ 3, 0,  0,  4, -1 ]

There is a also a handy function which computes the basis for the q-ary lattice and (optionally) applies LLL. If you are only interested in the shortest vector found, you can throw away the others:

In [120]:
from kryptools import q_ary_lattice

x = q_ary_lattice(K, lll = True)[:,0]
assert not A * x
x

[ 1 ]
[ 2 ]
[ 0 ]
[ 2 ]
[ 3 ]

Furthermore, there is another function which cobines all these steps and solves the SIS using LLL reduction. Of course this will only work for small lattices and in general a better lattice reduction algorithm like [BKZ](https://en.wikipedia.org/wiki/Korkine–Zolotarev_lattice_basis_reduction_algorithm) would be needed. There is also a variant which does a brute-force search (which however is rather slow):

In [121]:
from kryptools import sis, sis_search

%time xl = sis(A)
%time xs = sis_search(A)

assert xl == xs

CPU times: user 959 µs, sys: 59 µs, total: 1.02 ms
Wall time: 1.02 ms
CPU times: user 1.14 ms, sys: 37 µs, total: 1.18 ms
Wall time: 1.18 ms


In order to solve the inhomogenous version $A x = b$ you can add the inhomogenous vector $b$ as a new column to $A$

In [122]:
b = Matrix([[gf.random()] for _ in range(3)])
A.append_column(b)
A

[ 12, 13, 1,  8, 16, 3 ]
[ 15, 12, 9, 15, 11, 8 ]
[  6, 16, 4,  9,  4, 4 ]

and proceed as before:

In [123]:
U = q_ary_lattice(A.kernel(), lll = True)
A.delete_columns(-1)
U

[ -1, 1,  1, 2,  1, -5 ]
[ -1, 2,  0, 0, -3,  0 ]
[  0, 0,  2, 3, -1,  1 ]
[  0, 2, -3, 3,  1,  1 ]
[  0, 3,  1, 0,  3,  0 ]
[ -3, 0, -2, 0,  2,  0 ]

Any vector which has $\pm 1$ as last entry gives a short solution to the ISIS problem. In this case you could subtract the third from the first vector:

In [124]:
U[:,0] - U[:,2]

[ -2 ]
[ -1 ]
[ -2 ]
[  3 ]
[ -1 ]
[ -1 ]

Another way is to solve the associated CVP. This is what the built-in function does:

In [125]:
from kryptools import isis, isis_search

%time xl = isis(A, b)
%time xs = isis_search(A, b)

assert xl == xs
xl.map(lambda x: x.sharp())
xl

CPU times: user 1.03 ms, sys: 85 µs, total: 1.11 ms
Wall time: 1.12 ms
CPU times: user 2.62 ms, sys: 66 µs, total: 2.69 ms
Wall time: 2.69 ms


[ -2 ]
[ -1 ]
[ -2 ]
[  3 ]
[ -1 ]

## Polynomials

A basic class for [polynomials](https://en.wikipedia.org/wiki/Polynomial), entered as a list of coefficients

In [126]:
from kryptools import Poly

p = Poly([2, 0, 2])
q = Poly([1, 2, 3, 4])

p, q

(2 x^2 + 2, 4 x^3 + 3 x^2 + 2 x + 1)

The usual arithmetic operations work as expected

In [127]:
p * q

8 x^5 + 6 x^4 + 12 x^3 + 8 x^2 + 4 x + 2

[Polynomial division](https://en.wikipedia.org/wiki/Polynomial_long_division) with remainder

In [128]:
d, m = q.divmod(p)

assert d * p + m == q
d, m

(2.0 x + 1.5, -2.0 x - 2.0)

Alternatively

In [129]:
q // p, q % p

(2.0 x + 1.5, -2.0 x - 2.0)

If you prefer a different ordering of the coefficients, you can use

In [130]:
Poly.print_reversed = False
p

2 + 2 x^2

In [131]:
Poly.print_reversed = True

If you prefer a more traditional way of entering polynomials you could use:

In [132]:
x = Poly([0, 1])

x**2 + (3 * x + 1)**2

10 x^2 + 6 x + 1

To avoid floats, we can work with fractions. Either by using `ring=Fraction` during creation or by applying `Fraction` to all coefficients (in place):

In [133]:
from fractions import Fraction

p.map(Fraction)
q.map(Fraction)

q.divmod(p)

(2 x + 3/2, -2 x - 2)

If you want the reduction of $q$ modulo $p$ to be made in place use:

In [134]:
q.mod(p)
q

-2 x - 2

The `gcd` and `lcm` can be computed as follows:

In [135]:
a = Poly([1, -2, 0, 1])
b = Poly([-1, 0, 1])
a.gcd(b), a.lcm(b)

(x - 1.0, x^4 + x^3 - 2.0 x^2 - x + 1.0)

The maximum of the absolute value of all coefficients can be obtained with

In [136]:
q.max()

2

and the sum of the absolute value of all coefficients with

In [137]:
q.sum()

4

If the coefficients are from a field, the polynomial can also be constructed from a list of x an y coordinates via [Lagrange interpolation](https://en.wikipedia.org/wiki/Lagrange_polynomial):

In [138]:
from kryptools import lagrange_interpolation

gf = Zmod(11)

x = gf([1, 2, 3, 4])
y = gf([0, 3, 1, 4])
p = lagrange_interpolation(x, y)
assert all([ p(xx) == yy for xx, yy in zip(x, y) ])
p

9 x^3 + 4 x^2 + 5 x + 4

Polynomials from a finite field can be factored

In [139]:
p.factor()

{9: 1, x + 10: 1, x + 4: 1, x + 6: 1}

and tested for irreducibility using a variant of [Rabin's irreducibility test](https://en.wikipedia.org/wiki/Factorization_of_polynomials_over_finite_fields#Rabin.27s_test_of_irreducibility) availble as `.rabin_test()`.

In [140]:
p.rabin_test()

False

If you want to work modulo a given polynomial, you can specify the modulus as a polynomial or a list of coefficients.

In [141]:
p = Poly([1, 2, 3], modulus = [1, 0, 0, 1], ring = Zmod(7))

p.inv()

5 x^2 + x

For the cyclic ring with modulus $x^n -1$ (used for example in NTRU), there is a shortcut

In [142]:
p = Poly([1, 2, 3], cyclic = 3, ring = Zmod(7))

p.inv()

2 x^2 + 4

If a negative number is given, then $x^n + 1$ (like for example in Kyber) is used as a modulus instead.

There is also some support for the case where the coefficients are from the ring $\mathbb{Z}_n$, however, operations requiring division will fail once a noninvertible element is hit.

In addition, there is a class for binary multivariate polynomials with limited functionality. Here a polynomial can be entered as a list of coefficients (with the monomials ordered by degree):

In [143]:
from kryptools import PolyBinMult

p = PolyBinMult([0, 1, 0, 1], 2)
p

x_0 + x_0 x_1

Addition, multiplication and evaluation is supported:

In [144]:
p([0, 1])

0

### Showcase: AES

Computations in a polynomial ring modulo a given polynomial, for example the [AES](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard) Polynomial $x^8+x^4+x^3+x+1$, can be done as follows:

In [145]:
from kryptools import Poly, Zmod

Z_2 = Zmod(2)
aes = [1, 1, 0, 1, 1, 0, 0, 0, 1]  # 1 + x + x^3 + x^4 + x^8


def PolyAES(c: list) -> "Poly":
    return Poly(c, ring=Z_2, modulus=aes)


p = PolyAES([1, 0, 1])
q = PolyAES([0, 1, 0, 0, 1, 1, 1, 1])

p * q

x^4 + x^2 + x + 1

For AES, bytes are identified with polynomials (in the Galois field $\mathbb{Z}_2[x]_{x^8+x^4+x+1}$) by identifying the individual bits with the coefficients of the polynomial. The nonlinear operation used in AES is given by computing the inverse in the above field.

In [146]:
def byte2poly(x: int, n: int = 8) -> "Poly":
    "Convert an integer to a polynomial."
    return PolyAES(list(reversed([int(d) for d in str(format(x, "0" + str(n) + "b"))])))


def poly2byte(p: "Poly") -> int:
    "Convert a polynomial to the corresponding integer."
    s = 0
    for i in reversed(p.coeff):
        s = s * 2 + int(i)
    return s


poly2byte(byte2poly(2).inv())

141

For example, a substitution table for the [AES S-box](https://en.wikipedia.org/wiki/Rijndael_S-box) can be computed as follows:

In [147]:
from kryptools import Matrix, circulant

A = circulant([1, 1, 1, 1, 1, 0, 0, 0])
b = Matrix([1, 1, 0, 0, 0, 1, 1, 0])


def aes_sbox(x: int | Poly) -> int:
    "AES S-box."
    if isinstance(x, int):
        x = byte2poly(x)
    if x:
        res = x.inv().coeff
        res += [0] * (8 - len(res))  # pad the result with zeros
    else:
        res = [0] * 8
    res = A * Matrix(res) + b
    s = 0
    for i in reversed(res):
        s = s * 2 + int(i)
    return s

# Variant defining the afine transformation via polynomials
#
#p1 = byte2poly(31)
#p1.modulus = None
#p2 = byte2poly(99)
#p2.modulus = None
#
#def aes_sbox(x: int | Poly) -> int:
#    "AES S-box."
#    if isinstance(x, int):
#        x = byte2poly(x)
#    if x:
#        x = x.inv()
#    x.modulus = [1] + [0] * 7 + [1]
#    x = x * p1 + p2
#    s = 0
#    for i in reversed(x.coeff):
#        s = s * 2 + int(i)
#    return s

print(Matrix(
    [[hex(aes_sbox(i + 16 * j)) for i in range(16)] for j in range(16)]
))  # Matrix(...) is just to get a nicer output

[ 0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30,  0x1, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76 ]
[ 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0 ]
[ 0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15 ]
[  0x4, 0xc7, 0x23, 0xc3, 0x18, 0x96,  0x5, 0x9a,  0x7, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75 ]
[  0x9, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84 ]
[ 0x53, 0xd1,  0x0, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf ]
[ 0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9,  0x2, 0x7f, 0x50, 0x3c, 0x9f, 0xa8 ]
[ 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2 ]
[ 0xcd,  0xc, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73 ]
[ 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e,  0xb, 0xdb ]
[ 0xe0, 0x

Alternatively you can use the built-in Galois field:

In [148]:
from kryptools import GF2_aes

x = 3
assert int(GF2_aes(x) ** -1) == poly2byte(byte2poly(x).inv())
assert int(GF2_aes(x).sbox()) == aes_sbox(x)

The [MixColumn](https://en.wikipedia.org/wiki/Rijndael_MixColumns) operation can be expressed as polynomial multiplication of polynomials with coefficients in $\mathrm{GF}(8)$ modulo $x^4+1$. Each column of the state matrix is regarded as a polynomial and multiplied with $3x^3+x^2+x+2$. The inverse is given by

In [149]:
Poly([2, 1, 1, 3], modulus=[1, 0, 0, 0, 1], ring=gf).inv()

8 x^3 + 8 x^2 + 2 x + 6

Now we can verify that the AES polynomial $x^8+x^4+x^3+x+1$ is irreducible:

In [150]:
a = Poly(aes, ring=Z_2)

a.rabin_test()

True

Similarly for the Ghash polynomial $x^{128} + x^7 + x^2 + x + 1$:

In [151]:
a = Poly([1, 1, 1, 0, 0, 0, 0, 1] + 120 * [0] + [1], ring=Z_2)

a.rabin_test()

True

Note that the number of irreducible monic polynomials of degree $k$ in the field $\mathbb{Z}_p$ is given by the [necklace polynomial](https://en.wikipedia.org/wiki/Necklace_polynomial).
For $\mathbb{Z}_2$ there are for example $30$ irreducible monic polynomials of degree $8$.

In [152]:
from kryptools import divisors, moebius_mu


def M(q: int, k: int) -> int:
    """Moreau's necklace-counting function."""
    return sum([moebius_mu(d) * q ** (k // d) for d in divisors(k)]) // k


M(2, 8)

30

We can list them as follows:

In [153]:
for i in range(256):
    a = Poly(
        list(reversed([int(d) for d in str(format(256 + i, "0" + str(9) + "b"))])),
        ring=Z_2,
    )
    if a.rabin_test():
        print(a)

x^8 + x^4 + x^3 + x + 1
x^8 + x^4 + x^3 + x^2 + 1
x^8 + x^5 + x^3 + x + 1
x^8 + x^5 + x^3 + x^2 + 1
x^8 + x^5 + x^4 + x^3 + 1
x^8 + x^5 + x^4 + x^3 + x^2 + x + 1
x^8 + x^6 + x^3 + x^2 + 1
x^8 + x^6 + x^4 + x^3 + x^2 + x + 1
x^8 + x^6 + x^5 + x + 1
x^8 + x^6 + x^5 + x^2 + 1
x^8 + x^6 + x^5 + x^3 + 1
x^8 + x^6 + x^5 + x^4 + 1
x^8 + x^6 + x^5 + x^4 + x^2 + x + 1
x^8 + x^6 + x^5 + x^4 + x^3 + x + 1
x^8 + x^7 + x^2 + x + 1
x^8 + x^7 + x^3 + x + 1
x^8 + x^7 + x^3 + x^2 + 1
x^8 + x^7 + x^4 + x^3 + x^2 + x + 1
x^8 + x^7 + x^5 + x + 1
x^8 + x^7 + x^5 + x^3 + 1
x^8 + x^7 + x^5 + x^4 + 1
x^8 + x^7 + x^5 + x^4 + x^3 + x^2 + 1
x^8 + x^7 + x^6 + x + 1
x^8 + x^7 + x^6 + x^3 + x^2 + x + 1
x^8 + x^7 + x^6 + x^4 + x^2 + x + 1
x^8 + x^7 + x^6 + x^4 + x^3 + x^2 + 1
x^8 + x^7 + x^6 + x^5 + x^2 + x + 1
x^8 + x^7 + x^6 + x^5 + x^4 + x + 1
x^8 + x^7 + x^6 + x^5 + x^4 + x^2 + 1
x^8 + x^7 + x^6 + x^5 + x^4 + x^3 + 1


### Showcase: NTRU

Creating a public/private key pair suitable for a baby version of [NTRU](https://en.wikipedia.org/wiki/NTRU)

In [154]:
from copy import copy
from random import sample, seed
seed(0)


def random_ternery_poly(n: int, dp: int, dm: int) -> "Poly":
    res = [0] * n
    smpl = sample(range(n), dp + dm)
    for i in smpl[:dp]:
        res[i] = 1
    for i in smpl[dp:]:
        res[i] = -1
    return Poly(res, cyclic = n)  # modulus = x^n - 1


n = 5
q = 41
p = 3
d = 2

gf_q = Zmod(q)
gf_p = Zmod(p)


found = False
while not found:
    f = random_ternery_poly(n, d+1, d)

    fp = copy(f)
    fp.map(gf_p)
    fq = copy(f)
    fq.map(gf_q)

    try:
        Fq = fq.inv()
        Fp = fp.inv()
        found = True
    except:
        pass

g = random_ternery_poly(n, d, d)
h = Fq * g
h
print("Private key:")
print(f"f = {f}, F_p = {Fp}")

print("Public key:")
print(f"(F_q = {Fp}, g = {g})")
print(f"h = {h}")

Private key:
f = x^4 + x^3 - x^2 - x + 1, F_p = - x^2 + 2
Public key:
(F_q = - x^2 + 2, g = - x^4 + x^3 + x^2 - x)
h = 21 x^2 - x + 21


### Showcase: Kyber

For a nice display we use:

In [155]:
from IPython.display import display, Math

Poly.print_latex = True  # make sure the representation in in latex format
from kryptools import Matrix, Poly
def print_matrix(name, M):
    display(Math(f"{name} = " + M.latex()))

Creating a public/private key pair suitable for a baby version of [Kyber](https://en.wikipedia.org/wiki/Kyber)

In [156]:
from IPython.display import display
from random import randint, seed
from kryptools import Zmod

try:
    from random import binomialvariate
except:  # for versions older then 3.12
    from random import random

    def binomialvariate(n: int = 1, p: float = 0.5) -> int:
        "Binomial random variable"
        return sum(random() < p for i in range(n))


seed(0)

p = 97
n = 4
k = 2
gf = Zmod(p)


def PolyKyber(c: list) -> "Poly":
    return Poly(c, ring=gf, cyclic = -n)  # modulus = x^n + 1


A = Matrix(
    [
        [PolyKyber([randint(0, p - 1) for _ in range(n)]) for i in range(k)]
        for j in range(k)
    ]
)
s = Matrix([PolyKyber([randint(-1, 1) for _ in range(n)]) for i in range(k)])
q = p // (4 * k * n)
e = Matrix(
    [PolyKyber([binomialvariate(2 * q) - q for _ in range(n)]) for i in range(k)]
)
b = A * s + e

print("Private key:")
print_matrix("s", s)

print("Public key:")
print_matrix("A", A)
print_matrix("b", b)

Private key:


<IPython.core.display.Math object>

Public key:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Computations can be speed up using the fact that the $x^{256}+1$ can be factored as $\prod_{j=0}^{128} (x^2 + 17^{2j+1})$ in $\mathbb{Z}_{3329}$:

In [157]:
from math import prod

p = 3329
gf = Zmod(p)

m = [Poly([pow(17, 2 * j + 1, p), 0, 1], ring=gf) for j in range(128)]
M = prod(m)
M

x^{256} + 1

Note that the factors $m_j(x)=x^2 + 17^{2j+1}$ are irreducible since $17$ is not a quadratic residue:

In [158]:
from kryptools import legendre_symbol

legendre_symbol(17, p)

-1

You could also use `Poly(kyber(256), ring = gf).factor()`, but this wil take a minute or two.

Hence a given polynomial $a(x)$ can be replaced by its remainders modulo $m_j(x)$ and all computations can be done with the remainders. The resulting polynomial can be constructed from the remainders using the Chinese Remainder Theorem. This is known as Number-Theoretic-Transform in this context.

In [159]:
def ntt(a: list["Poly"], m: list["Poly"]) -> "Poly":
    """Solve given linear polynomial congruences x % m[j] == a[j] using the Chinese Remainder Theorem."""
    l = len(a)
    assert len(m) == l, "The lists of numbers and modules must have equal length."
    M = prod(m)
    Mi = [M.divmod(m[i])[0] for i in range(l)]
    for i in range(l):
        Mi[i].modulus = M.coeff
    MNi = [Mi[i] * Mi[i].inv(m[i]) for i in range(l)]
    return sum([(a[i] * MNi[i]).divmod(M)[1] for i in range(l)])


a = Poly([1, 2, 3, 4, 5, 6], ring=gf)
aj = [a.divmod(m[j])[1] for j in range(128)]

ntt(aj, m)

6 x^5 + 5 x^4 + 4 x^3 + 3 x^2 + 2 x + 1

### Showcase: AKS primality test

Here is a proof of concept implementation of the [AKS primality test](https://en.wikipedia.org/wiki/AKS_primality_test):

In [160]:
from kryptools import perfect_power, order, euler_phi, Poly, Zmod
from math import gcd, log2, sqrt, floor
from kryptools import is_prime

def aks_test(n: int, verbose = False) -> bool:
    "Test if `n` is prime using the AKS primality test."
    def vprint(s: str, end: str = "\n") -> None:
        if verbose:
            print(s, end = end)
    if n < 2:
        return False
    # Step 1: Test if n is a perfect power
    res = perfect_power(n)
    if res:
        vprint(f"Perfect power: n = {res[0]}^{res[1]}")
        return False
    # Step 2
    r = 2
    l2n2 = log2(n)**2
    while True:
        g = gcd(r,n)
        if g > 1:
            if r < n:
                vprint(f"Common factor found while searching for r: {g}")
                return False
        elif order(n, r) > l2n2:
            break
        r += 1
    vprint(f"r = {r}")
    # Step 4
    if n <= r:
        print(f"Condition n <= r is met.")
        return True
    # Step 3
    for a in range(2, min(r,n-1)+1):
        if not a % n:
            vprint(f"Common factor found while testing a's: {a}")
            return False
    # Step 5
    vprint("Checking (x-a)^n = x^n - a: ", end ="")
    Z_n = Zmod(n)
    modulus = [-1] + [0]* (r-1) + [1]
    for a in range(2, floor(sqrt(euler_phi(r)) * log2(n)) + 1):
        vprint(f"{a} ", end="")
        # Check (x-a)^n = x^n - a in Z_n[x]_{x^r-1}
        pol = Poly([a, 1], ring = Z_n, modulus = modulus)**n - Poly([0, 1], ring = Z_n, modulus = modulus)**n - Poly([a], ring = Z_n, modulus = modulus)
        if pol:
            vprint("")
            return False
    vprint("")
    return True

n = 31
assert aks_test(n, verbose = True) == is_prime(n)

r = 29
Checking (x-a)^n = x^n - a: 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 


## Eliptic curves

To declare an [elliptic curve](https://en.wikipedia.org/wiki/Elliptic_curve) in Weierstrass normal form use

In [161]:
from kryptools import EC_Weierstrass

ec = EC_Weierstrass(239, 3, 1)
ec.info()
print("Order:", ec.order())

Weierstrass curve y^2 = x^3 + 3 x + 1 over Z_239.
Order: 258


The order is computed using the Shanks-Maestre algorithm.

Basic arithmetic

In [162]:
P = ec.random()
Q = ec.random()
O = ec.inf()  # point at infinity
P + Q

(113, 91)

To check if a point is on the curve use

In [163]:
P in ec

True

To compute the order of a point use

In [164]:
assert Q.order() * Q == O
Q.order()

129

The $n$'th division polynomial $\Psi_n(x,y)$ will anihilate all points whose order is a multiple of $n$:

In [165]:
P.psi(P.order())

0

To compute a discrete log (Pohlig-Hellman reduction and Shank's baby step/giant step algorithm) use

In [166]:
k = 12
R = k * Q
assert R.dlog(Q) == k % Q.order()

### Showcase: Bitcoin curve

Setting the NIST curve [`secp256k1`](https://en.bitcoin.it/wiki/Secp256k1)

In [167]:
p = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 - 1
m = 115792089237316195423570985008687907852837564279074904382605163141518161494337  # order of the curve
ec = EC_Weierstrass(p, 0, 7, m)
ec.hex = True  # display points in hex form
ec.short = True  # display points in short form
G = ec(
    "02 79BE667E F9DCBBAC 55A06295 CE870B07 029BFCDB 2DCE28D9 59F2815B 16F81798"
)  # generator

Generate a (pseudorandom) key pair

In [168]:
secret_key = randint(2, m - 1)
public_key = secret_key * G
secret_key, public_key

(23529166972251975810298992610444093284962271729068871731505428628592805361023,
 0x03449ef2ed30a8deb96e584a08c329adbf1be87ce40f1a0e7b4e86178682c41a9c)

## Linear codes

A [linear code](https://en.wikipedia.org/wiki/Linear_code) of length $n$ over a finite field $\mathbb{K}$ is a linear subspace of $\mathbb{K}^n$. The dimension of the code is usually denoted by $k$ and one speaks of an $[n,k]$ code. A matrix $G$ whose column vectors form a basis for the code is known as generator matrix of the code.

In [169]:
from kryptools import Zmod, Matrix
Z_2 = Zmod(2)

G = Matrix([[0, 1, 1], [1, 1, 0]], ring = Z_2)
G

[ 0, 1, 1 ]
[ 1, 1, 0 ]

Computing the row reduced echelon form of $G$ will give you a simpler basis, still generating the same code. In addition, if you permute the rows you get an equivalent code whose generator matrix is of the form $G=(\mathbb{I}_k, M)$. This is known as left standard or systematic form.

In [170]:
GG = G.left_standard_form()
GG

[ 1, 0, 1 ]
[ 0, 1, 1 ]

Note that when computing the row reduced echelon form, the list of pivot and nonpivot colums are stored and this gives you the column perturbation used to obtain the left standard form:

In [171]:
GG = G.rref()
GG.permute_columns(GG.pivotcols + GG.nonpivotcols)
assert GG == G.left_standard_form()

Similarly you could compute the corresponding permutation matrix:

In [172]:
P = G.eye(G.cols)
P.permute_columns(GG.pivotcols + GG.nonpivotcols)

assert G.rref() * P == G.left_standard_form()

A matrix whose kernel equals the code is known as parity check matrix for the code. If the generator matrix is in left standard form $G=(\mathbb{I}_k, M)$, one can easily verify, that a parity check matrix is $H=(-M^T, \mathbb{I}_{n-k})$. You can compute the transposed kernel of $G$:

In [173]:
H = G.kernel().transpose()
H

[ 1, 1, 1 ]

Now you can encode given $k$ bits using

In [174]:
x = Matrix([0, 1])
y = GG.transpose() * x
list(y)

[0, 1, 1]

To check that no transmission errors have occured you apply $H$ (the result is known as known as syndrome)

In [175]:
H * y

0

To decode you solve the corresponding system

In [176]:
list(GG.transpose().solve(y))

[0, 1]

Suppose some errors occured during the transmission of $y$ and $yy$ was recived. The number of errors (i.e. changed positions) is knonw as [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance)

In [177]:
from kryptools import hamming_dist

yy = Matrix([1, 1, 1], ring = Z_2)
hamming_dist(y, yy)

1

In this case the error can be dedected with the parity check matrix:

In [178]:
H * yy

1

The number of errors which can always be dedected equals the minimal Hamming distance between code words, which by linearity equals the minimal distance of all nontrivial code words from zero. The following [Hamming code](https://en.wikipedia.org/wiki/Hamming_code)

In [179]:
def Hamming2(h: int) -> Matrix:
    "Parity check matrix for the binary Hamming code."
    out = []
    for i in range(1,2**h):
        out.append([int(d) for d in str(format(i, "0" + str(h) + "b"))])
    return Matrix(out, ring = Z_2).transpose()


H = Hamming2(3)
G = H.kernel().transpose()
G = G.rref()
assert not H * G.transpose()
G

[ 1, 0, 0, 0, 0, 1, 1 ]
[ 0, 1, 0, 0, 1, 0, 1 ]
[ 0, 0, 1, 0, 1, 1, 0 ]
[ 0, 0, 0, 1, 1, 1, 1 ]

has minimal distance $d=3$. Hence it cannot only dedect $d-1 = 2$ errors but even correct $\lfloor (d-1)/2 \rfloor = 1$ error.

In [180]:
x = [ 0, 1, 1, 0]
y = list(G.transpose() * Matrix(x))
y[2] += 1 # add one error
y

[0, 1, 0, 0, 0, 1, 1]

The position at which the error occured can be easily found! In fact, since the columns of the parity check matrix are precisely the binary digits of the row number (starting to count at $1$), the syndrom $y H^T$ tells us precisely the error location:

In [181]:
s = H * Matrix(y)
error_position = sum( int(si) * 2**i for i, si in enumerate(reversed(s)) )
if error_position:
    print(f"Correcting error at index {error_position-1}.")
    y[error_position-1] += 1
y = list(map(int, y))
assert y[:4] == x
y[:4]

Correcting error at index 2.


[0, 1, 1, 0]

Another classical code is the binary [Golay code](https://en.wikipedia.org/wiki/Binary_Golay_code). In fact, Golay observed the identity

In [182]:
from math import comb

sum(comb(23, i) for i in range(3+1)) == 2**(23-12)

True

indicating that there might be a binary $[23,12,7]$ code attaining equality in the [sphere-packging](https://en.wikipedia.org/wiki/Hamming_bound) bound. And he indeed found a corresponding code:

In [183]:
from kryptools import Matrix, eye
from kryptools.la import rotate

G = eye(12).matrix
for n in range(11):
    G[n] += rotate([1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0], n)
G[11] += [1] * 11
G = Matrix(G, ring=Z_2)
G

[ 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0 ]
[ 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1 ]
[ 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1 ]
[ 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0 ]
[ 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1 ]
[ 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1 ]
[ 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1 ]
[ 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0 ]
[ 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0 ]
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0 ]
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1 ]
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ]

We first check the minimum distance:

In [184]:
from kryptools import GF2, hamming_dist

def minimum_distance(G: Matrix) ->int:
    "Compute the minimum distance of a binary linear code with given generator matrix."
    d =  G.cols
    zero = Matrix([0] * d)
    for x in GF2(G.rows).star():
        d = min(d, hamming_dist((G.transpose() * Matrix(x.bits())).applyfunc(int), zero))
    return d

d_min = minimum_distance(G)
print(f"Parameters [n,k,d] = [{G.cols},{G.rows},{d_min}]")

Parameters [n,k,d] = [23,12,7]


Hence it cannot only dedect $d-1 = 6$ errors but even correct $\lfloor (d-1)/2 \rfloor = 3$ errors. To correct the errors one has to find the codeword with minimal distance to the recieved word.

We first create some sample vectors and add some errors.

In [185]:
from random import randint, sample

t = (d_min - 1)//2  # number of errors the code can correct
assert t >= 1

x = Matrix([randint(0,1) for _ in range(G.rows)], ring = Z_2)
y = G.transpose() * x
e = y.zeros()
for i in sample(range(G.cols), t):
    e[i] += Z_2(1)
yy = y + e

To find the errors one computes the syndrome which only depends on the error.

In [186]:
H = G.kernel().transpose()
list(H * yy)

[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1]

Then, given a precomputed table with the syndromes of all possible errors, one can determine the error and decode the corrected word:

In [187]:
from itertools import combinations

syndromes = {}
for s in range(t+1):
    for comb in combinations(range(G.cols), s):
        ee = [Z_2(0)] * G.cols
        for i in comb:
            ee[i] = Z_2(1)
        s = tuple((H * Matrix(ee)).applyfunc(int))
        if not s in syndromes:
            syndromes[s] = ee

ee = Matrix(syndromes[tuple((H * yy).applyfunc(int))])
assert e == ee
list(yy - ee)[:12]

[1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1]

Alternatively one can use [information set decoding](https://en.wikipedia.org/wiki/Decoding_methods). To this end one selectes $k$ linearly independent equations from the system $G^T x  =y$ and hopes that no errors occured within the selected bits from $y$. We do not bother to check the selected equations for linear independence and solve right away, as this will not affect the outcome.

In [188]:
from random import sample

count = 0
found = False
while not found:
    count += 1
    Gt = G.zeros(G.rows)  # template to hold the selected columns
    yt = yy.zeros(G.rows, 1)  # template to hold the corresponding bits from y
    for i, j in enumerate(sample(range(G.cols), G.rows)):
        Gt[:,i] = G[:,j]
        yt[i] = yy[j]
    xx = Gt.transpose().solve(yt) # try to solve the selection
    if xx is None:  # no soluton
        continue
    yt = G.transpose() * xx
    if hamming_dist(yy, yt) <= t:  # check if the solution has at most t errors
        found = True

print(f"Solution found with {count} trys.")
assert x == xx
list(x)

Solution found with 8 trys.


[1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1]

Note that it is easy to add a parity check bit to the code (in this case known as the extended binary Golay code):

In [189]:
G.append_column([ sum(G.matrix[j]) for j in range(G.rows) ])
print(f"Parameters [n,k,d] = [{G.cols},{G.rows},{minimum_distance(G)}]")
G

Parameters [n,k,d] = [24,12,8]


[ 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1 ]
[ 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1 ]
[ 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1 ]
[ 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1 ]
[ 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1 ]
[ 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1 ]
[ 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1 ]
[ 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1 ]
[ 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1 ]
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1 ]
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1 ]
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0 ]

Or you can also go in the other direction and shorten the code by $s$ Bits:

In [190]:
s, k = 2, G.rows
del G[k-s:k, :]
del G[:, k-s:k]
print(f"Parameters [n,k,d] = [{G.cols},{G.rows},{minimum_distance(G)}]")
G

Parameters [n,k,d] = [22,10,8]


[ 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1 ]
[ 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1 ]
[ 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1 ]
[ 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1 ]
[ 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1 ]
[ 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1 ]
[ 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1 ]
[ 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1 ]
[ 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1 ]
[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1 ]

There is also a convenient class for binary code, which construct a code from a generator matrix (or from a parity check matrix if the option `parity_check` is set).

In [191]:
from kryptools import BinaryCode

bc = BinaryCode(G)
bc

Binary linear systematic [22, 10, 8] code.

The minimum distance is computed automatically using the Brouwer-Zimmermann algorithm (which might take a while) and can be turned off using the option `minimum_distance`. Encoding and decoding is done using syndrom decoding as shown above.

In [192]:
bc.encode([0, 1, 0, 0, 0, 1, 1, 1, 0, 0])

[0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0]

In [193]:
bc.decode([1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0])

[0, 1, 0, 0, 0, 1, 1, 1, 0, 0]

A code is per default replaced by an equivalent code in systematic form unless the option `systematic` is set to `False`. Information set decoding, as described above, is avalaible as `is_decode`.

The minimumdistance (if not computed during creation) and the error correction capability (the number of errors the code can correct) can be obtained with

In [194]:
bc.minimum_distance(), bc.correction_cap()

(8, 3)

The generator matrix is given by

In [195]:
bc.G

[1000000000110111000101]
[0100000000101110001011]
[0010000000011100010111]
[0001000000111000101101]
[0000100000110001011011]
[0000010000100010110111]
[0000001000000101101111]
[0000000100001011011101]
[0000000010010110111001]
[0000000001101101110001]

and the parity check matrix is given by

In [196]:
bc.H

[1101110001100000000000]
[1011100010010000000000]
[0111000101001000000000]
[1110001011000100000000]
[1100010110000010000000]
[1000101101000001000000]
[0001011011000000100000]
[0010110111000000010000]
[0101101110000000001000]
[1011011100000000000100]
[0110111000000000000010]
[1111111111000000000001]

Both are stored as a `BinaryMatrix` and you can wrap `Matrix( )` around it to get a usual matrix with entries from $\mathbb{Z}_2$.

There are predefined classes for some classical binary codes: `GolayCode()`, `HammingCode(n)`, `ReedMullerCode(s, m)`

A particulary important class for cryptography are Goppa codes. A [binary Goppa code](https://en.wikipedia.org/wiki/Binary_Goppa_code) is defined over the polynomial ring $GF(2^m)[x]_{g(x)}$, where $g(x)$ is some given polynomial from $GF(2^m)[x]$. Then given a list of points $\alpha_0, \dots, \alpha_{k-1} \in GF(2^m)$ which must not be zeros of $g$ the code is defined by all $y\in \mathbb{Z}_2^k$ for which
$$
\sum_{j=0}^{k-1} y_j (x-\alpha_j)^{-1} = 0 \pmod{g(x)}.
$$

In [197]:
from kryptools import GF2, Poly, GoppaCode

gf = GF2(4)
g = Poly([2, 0, 0, 1], ring = gf)
alpha = list(gf)

goppa = GoppaCode(gf, g, alpha)
goppa

Binary irreducible Goppa [16, 4] code over GF(2^4) with polynomial g(x) = x^3 + 2.

If at most $t=\deg(g)$ errors occured, decoding can be done efficiently using Patterson's algorithm.

In [198]:
x = [1, 0, 1, 1]
y = goppa.encode(x)
assert goppa.decode(y) == x
y

[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1]

In [199]:
yy = [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1]
assert hamming_dist(y, yy) <= g.degree()
assert goppa.decode(yy) == x

Another important class are [cyclic codes](https://en.wikipedia.org/wiki/Cyclic_code). Given a finite field $\mathbb{K}$ a cyclice code is given by an ideal in $\mathbb{K}[x]_{x^n-1}$, where meassages and code words are identified with polynomials.
Since $\mathbb{K}[x]_{x^n-1}$ is a principal ideal domain, every code is generated by a polynomial. The smallest polynomial is the generator polynomial of the code an must be a factor of $x^n-1$.

In [200]:
from kryptools import CyclicCode, Poly

gf = Zmod(2)
n = 4
g = Poly([1, 1], ring = gf)
cc = CyclicCode(n, g)
cc

Cyclic systematic [4, 3] code over GF(2) with generator polynomial g(x) = x + 1.

The generator matrix is

In [201]:
G = cc.generator_matrix()
G

[ 1, 1, 0, 0 ]
[ 1, 0, 1, 0 ]
[ 1, 0, 0, 1 ]

and the parity check matrix is

In [202]:
H = cc.check_matrix()
assert not H * G.transpose()
H

[ 1, 1, 1, 1 ]

To encode a message use (note the message vector will automatically be converted to the Galois field):

In [203]:
x = [1, 1, 0]
y = cc.encode(x)
y

[0, 1, 1, 0]

This is normally done by multiplying the message with the generator polynomial. If the systematic form of the encoding is used, the message polynomial is first muliplied by $x^{\deg(g)}$ and then the remainder of the division with $g$ is subtracted such that the message can be read off from the highest coefficients of the encoded polynomial.

To decode use (currently only works for code words):

In [204]:
assert cc.decode(y) == gf(x)

Finally, there is support for [Reed-Solomon codes](https://en.wikipedia.org/wiki/Reed–Solomon_error_correction), which are a special class of cyclic codes. Here $n$ is chosen to be the order of $\mathbb{K}^*$ such that $x^n-1$ completly factors, having all elements from $\mathbb{K}^*$ as zeros. In particular, since $\mathbb{K}^*$ we can find an generator $\alpha$ such that $\mathbb{K}^* = \{ \alpha^0, \alpha^1, \alpha^2, \dots, \alpha^{n-1}\}$. The genrator is choosen automatically unless explicitely specified.

In [205]:
from kryptools import ReedSolomonCode, Zmod

gf = Zmod(13)
k = 3

rsc = ReedSolomonCode(k, gf)
rsc

Reed-Solomon [12, 3, 10] code over GF(13).

A generator matrix and a parity check matric can be obtained as before. However, encoding is done differently, by regarding the message as a polynomial and evaluation this polynomial on all elements of $\mathbb{K}^*$ (in the order given by the powers of $\alpha$):

In [206]:
rsc.encode([1, 2, 3])

[6, 4, 5, 1, 8, 4, 2, 9, 2, 8, 9, 6]

Decoding can be done by evaluation the code word at all the inverse powers of $\alpha$ and muliplying the value by $-1$.

In [207]:
rsc.decode([6, 4, 5, 1, 8, 4, 2, 9, 2, 8, 9, 6])

[1, 2, 3]

The advantage is that as long as $k$ correct values of the code word are known, the original message can be reconstructed using Lagrange interpolation. However, even if the location of the errors are not known, $\lfloor (n-k)/2 \rfloor$ errors can still be corrected.

In [208]:
rsc.decode([6, 4, 5, 2, 8, 4, 2, 9, 3, 8, 8, 6])

[1, 2, 3]

If you restrict a Reed-Solomon code over $GF(2^m)$ to $GF(2)$ you get a binary narrow sense primitive [BCH](https://en.wikipedia.org/wiki/BCH_code) code. For example to get a BCH code over $GF(2^4)$, that is code length $n=2^4-1 =15$ whose minimum distance is at least $5$ use:

In [209]:
from kryptools import GF2, BCHCode

bch = BCHCode(15, 5)
bch

BCH [15, 7] code over GF(2) with generator polynomial g(x) = x^8 + x^7 + x^6 + x^4 + 1.

The underlying RS code used for error correction is:

In [210]:
bch.rsc

Reed-Solomon [15, 11, 5] code over GF(16).

In [211]:
from random import randint

x = [1, 0, 0, 1, 1, 0, 1]

y = bch.encode(x)
for _ in range(2): # the code can correct at least (5-1) // 2 = 2 errors
    i = randint(0, bch.n - 1)
    y[i] += GF2(1)(1)

assert list(map(int, bch.decode(y))) == x

## DES

The DES module implements a basic DES-type Feistel network plus the corresponding key-schedule. Specifying the corresponding permutations and sboxes, one can get instances of S-DES and DES.

For SDES the two classes are `SDESKeySchedule` and `SDESCipher`. The first class is used to compute the round keys. Internally everything is represented as a list of the individual bits. This is the state to begin with:

In [212]:
from kryptools import SDESKeySchedule

ks = SDESKeySchedule(34)
ks

[0, 1, 0, 0, 0, 0, 0, 1, 0, 0]

If you prefer the output as a hex number, you can use the following option:

In [213]:
SDESKeySchedule.print_hex = True
print(ks)
SDESKeySchedule.print_hex = False

104


Using the `next()` method you can generate the keys one after the other

In [214]:
ks.next()

[0, 0, 1, 0, 0, 0, 0, 0]

You can also generate all round keys (in the case of SDES there are just two) at once. But you might want to reset the state in case you already generated some keys.

In [215]:
ks.reset()
keys = ks.gen_keys()
keys

[[0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 1, 0]]

There is also a handy function for converting the list of bits back to an integer:

In [216]:
from kryptools import list2int

list2int(keys[0])

32

Of course there is also a function which converts an integer to a list of bits with fixed length given by the second argument:

In [217]:
from kryptools import int2list

int2list(32, 8)

[0, 0, 1, 0, 0, 0, 0, 0]

Next, there is the `SDESCipher` class. It gives you access to all individual operations

In [218]:
from kryptools import SDESCipher

x = b"\x01"  # the message to be encrypted
sdes = SDESCipher()
sdes(x)  # set up the cipher

print(sdes)
sdes.ip()  # apply the initial permutation
print(sdes)

sdes.round(keys[0])  # apply one round with the given round key
print(sdes)

sdes.round(keys[1], last=True)  # The last round will not switch the two halfes
print(sdes)

sdes.fp()  # apply the final permutation
# print(sdes)

int(sdes)

00000001
00000100
01001110
00111110


185

Of course you can also encrypt/decrypt individual blocks. The result is a single byte and you can use the following trick to get the result as an integer:

In [219]:
sdes = SDESCipher(34)
sdes.encrypt(x)[0]

185

Note that it is easy to change the individual components of the Feistel cipher. For example if you change the schedule of left shifts for the key schedule for S-DES from `ROT = (1, 2)` to (say) `ROT = (1, 2, 2, 1)`, you will get a variant of S-DES with 4 rounds. This is conveniant if you want to apply linear or differential cryptoanalysis (which is kind of pointless for a Feistel network with only two rounds).

In [220]:
sdes.key_schedule.ROT = (1, 2, 2, 1)
sdes.set_key(34)  # to use the new key schedule
sdes.encrypt(x)[0]

107

Of course the same works with DES. Here are some test vectors from  the [Handbok of Applied Cryptography](https://cacr.uwaterloo.ca/hac/) (Example 7.86):

In [221]:
from kryptools import DESCipher

key = 0x0123456789ABCDEF.to_bytes(8, "big")
x = 0x4E6F772069732074.to_bytes(8, "big")
y = 0x3FA40E8A984D4815.to_bytes(8, "big")

des = DESCipher(key)
des(x).encrypt()
print(des)
assert bytes(des) == y
assert des.decrypt() == x

3fa40e8a984d4815


Here are the sample round outputs from [NBS SP500-20](https://doi.org/10.6028/NBS.SP.500-20e1980) (Figure 4 on p9):

In [222]:
key = 0x10316E028C8F3B4A.to_bytes(8, "big")
x = 0x0000000000000000.to_bytes(8, "big")
y = 0x82DCBAFBDEAB6602.to_bytes(8, "big")


des = DESCipher(key)
des(x)
for i in range(16):
    fbox = list2int(des.f_box(des.right(), des.keys[i]))
    roundkey = list2int(des.keys[i])
    des.round(des.keys[i], last=(i == 15))
    res = str(des)
    print(
        f"round {i:02d}: {res[:8]} {res[8:]}  (key= {roundkey:012x}, fbox= {fbox:08x})"
    )
des.fp()
assert bytes(des) == y
des

round 00: 00000000 47092b5b  (key= 4220438e6f23, fbox= 47092b5b)
round 01: 47092b5b 53f372af  (key= 182235355963, fbox= 53f372af)
round 02: 53f372af 9f1d158b  (key= 851408c68876, fbox= d8143ed0)
round 03: 9f1d158b 8109cbee  (key= 4202e4c5afdc, fbox= d2fab941)
round 04: 8109cbee 60448698  (key= 98d8003996d9, fbox= ff599313)
round 05: 60448698 29ebb1a4  (key= 00236a5bd427, fbox= a8e27a4a)
round 06: 29ebb1a4 620cc3a3  (key= a054050e6dac, fbox= 0248453b)
round 07: 620cc3a3 deeb3d8a  (key= 410b40a879d5, fbox= f7008c2e)
round 08: deeb3d8a a1a0354d  (key= 620078ef2871, fbox= c3acf6ee)
round 09: a1a0354d 9f0303dc  (key= 8cc004a3eb5a, fbox= 41e83e56)
round 10: 9f0303dc fd898ee8  (key= 020b1a359716, fbox= 5c29bba5)
round 11: fd898ee8 2d1ae1dd  (key= 2c3021dd04e6, fbox= b219e201)
round 12: 2d1ae1dd cbc829fa  (key= 830c484ceacd, fbox= 3641a712)
round 13: cbc829fa b367dec9  (key= 48629032f4dd, fbox= 9e7d3f14)
round 14: b367dec9 3f6c3efd  (key= 149d08ab95a3, fbox= f4a41707)
round 15: 5a1e5228 3f6c3e

82dcbafbdeab6602

## AES

The AES module implements two classes: `AESKeySchedule` and `AESCipher`. The first class is used to compute the round keys. For example, here are the first two rounds of the key schedule with details:

In [223]:
from kryptools import AESKeySchedule

key = 0x2B7E151628AED2A6ABF7158809CF4F3C.to_bytes(16, "big")
ks = AESKeySchedule(key)
ks.showtmp = True

print(ks)
print("------------------")
for _ in range(2):
    print(f"----Round {ks.round+1}-------")
    ks.next()
    print(ks)

[ 2b, 28, ab, 09 ]
[ 7e, ae, f7, cf ]
[ 15, d2, 15, 4f ]
[ 16, a6, 88, 3c ]
------------------
----Round 1-------
Start    :[09, cf, 4f, 3c]
RotWord  :[cf, 4f, 3c, 09]
SubWord  :[8a, 84, eb, 01]
AddConst :[8b, 84, eb, 01]
[ a0, 88, 23, 2a ]
[ fa, 54, a3, 6c ]
[ fe, 2c, 39, 76 ]
[ 17, b1, 39, 05 ]
----Round 2-------
Start    :[2a, 6c, 76, 05]
RotWord  :[6c, 76, 05, 2a]
SubWord  :[50, 38, 6b, e5]
AddConst :[52, 38, 6b, e5]
[ f2, 7a, 59, 73 ]
[ c2, 96, 35, 59 ]
[ 95, b9, 80, f6 ]
[ f2, 43, 7a, 7f ]


Using the `AEScipher` class you can watch how the state changes during individual rounds:

In [224]:
from kryptools import AESCipher

verbose = True
key = 0x2B7E151628AED2A6ABF7158809CF4F3C.to_bytes(16, "big")
x = 0x3243F6A8885A308D313198A2E0370734.to_bytes(16, "big")

state = AESCipher(key)
state(x)  # initialize the state
print(state)
print("------------------")
print("----AddKey--------")
state.add_key(state.keys[0])
print(state)
for r in range(1, 2):
    print(f"----Round {r}-------")
    if verbose:
        print("----SubBytes------")
    state.sub_bytes()
    if verbose:
        print(state)
        print("----ShiftRows-----")
    state.shift_rows()
    if verbose:
        print(state)
    if r != 10:
        if verbose:
            print("----MixColumns----")
        state.mix_columns()
        if verbose:
            print(state)
    if verbose:
        print("----AddKey--------")
    state.add_key(state.keys[r])
    print(state)

[ 32, 88, 31, e0 ]
[ 43, 5a, 31, 37 ]
[ f6, 30, 98, 07 ]
[ a8, 8d, a2, 34 ]
------------------
----AddKey--------
[ 19, a0, 9a, e9 ]
[ 3d, f4, c6, f8 ]
[ e3, e2, 8d, 48 ]
[ be, 2b, 2a, 08 ]
----Round 1-------
----SubBytes------
[ d4, e0, b8, 1e ]
[ 27, bf, b4, 41 ]
[ 11, 98, 5d, 52 ]
[ ae, f1, e5, 30 ]
----ShiftRows-----
[ d4, e0, b8, 1e ]
[ bf, b4, 41, 27 ]
[ 5d, 52, 11, 98 ]
[ 30, ae, f1, e5 ]
----MixColumns----
[ 04, e0, 48, 28 ]
[ 66, cb, f8, 06 ]
[ 81, 19, d3, 26 ]
[ e5, 9a, 7a, 4c ]
----AddKey--------
[ a4, 68, 6b, 02 ]
[ 9c, 9f, 5b, 6a ]
[ 7f, 35, ea, 50 ]
[ f2, 2b, 43, 49 ]


## Block Ciphers

The `BlockCipher` class provides various mode of operations for a block cipher. One way of using it is to provide the encryption/decryption function for one block. An instance gets initialized with the secret key and a mode of operation (`ecb`, `cbc`, etc). If the chosen mode requires padding, PKCS#7 will be used. Also a random IV (initial vector) will be choosen automatically and added to the cipher text as the first block. Accordingly, decryption expects the IV as first block for such modes.

In [225]:
from kryptools import BlockCipher, bytexor


class XORBlockCipher(BlockCipher):
    blocksize = 2

    def encrypt_block(self, b: bytes):
        return bytexor(b, self.key)

    def decrypt_block(self, b: bytes):
        return bytexor(b, self.key)


xor2 = XORBlockCipher(b"az", mode="cbc")
text = b"test1"
ctext = xor2.encrypt(text)
assert text == xor2.decrypt(ctext)
ctext

b'\x04\xe2\x11\xfd\x03\xf3S\x88'

Another way of using it is to provide a class for the cipher which supports an encrypt/decrypt method.

In [226]:
class CaesarCipher:
    "Caesar cipher"

    def __init__(self, key: bytes):
        self.key = key

    def encrypt(self, text: bytes):
        "Encrypt one block."
        ctext = bytearray(b"\x00" * len(self.key))
        for i in range(len(self.key)):
            ctext[i] = (text[i] + self.key[i]) % 256
        return bytes(ctext)

    def decrypt(self, ctext: bytes):
        "Decrypt one block."
        text = bytearray(b"\x00" * len(self.key))
        for i in range(len(self.key)):
            text[i] = (ctext[i] - self.key[i]) % 256
        return bytes(text)


class CaesarBlockCipher(BlockCipher):
    blocksize = 4

    def set_key(self, key: bytes):
        if len(key) != self.__class__.blocksize:
            raise ValueError(f"Key must be {self.__class__.blocksize} bytes.")
        self.key = key
        self.cipher = CaesarCipher(key)


ceasar = CaesarBlockCipher(key=b"abcd", mode="cbc")
text = b"test123"
ctext = ceasar.encrypt(text)
assert text == ceasar.decrypt(ctext)
ctext

b'\xf9\xe9Q\x9a\xee\xee\x85R@>\x19\xb7'

You can also compute a cbc mac

In [227]:
ceasar.mac_cbc(text)

b'EWH='

or a CMAC (the blocksize (in bytes) must be 1,2,4,8 or 16)

In [228]:
ceasar.mac_cmac(text)

b'\xc2\xde\xcb\xa9'

Of course you can use the classes for DES or AES from above.

In [229]:
from kryptools import AESCipher, BlockCipher


class AESBlockCipher(BlockCipher):
    blocksize = 16

    def set_key(self, key: bytes):
        if not isinstance(key, bytes) or len(key) != self.__class__.blocksize:
            raise ValueError(f"Key must be {self.__class__.blocksize} bytes.")
        self.key = key
        self.cipher = AESCipher(key)

For example you can use Galois counter mode with associated authenticated data:

In [230]:
key = bytes.fromhex("36f18357be4dbd77f050515c73fcf9f2")
text = b"Test message"
aad = b"ab"

aes = AESBlockCipher(key)
ctext = aes.encrypt_gcm(text, aad=aad)
assert aes.decrypt_gcm(ctext, aad=aad) == text
ctext

b"\x93'\xb8A\xd8G'\xad\x87\xa8\xe0(YB\x99l\xe0>\xa4-XD\xd2\x03\xce.l\xe1\xf6M\xde\x05\xc0Kl\x8a\xd1\xcc.\x89"

## Hash Functions

First of all there is [SHA-1](https://en.wikipedia.org/wiki/SHA-1)

In [231]:
from kryptools import SHA1

sha1 = SHA1()
sha1(b'abc').hex()

'a9993e364706816aba3e25717850c26c9cd0d89d'

A simple prefix HMAC based on SHA-1 can be computed as follows:

In [232]:
message = b'An authemtic message.'
key = b'12345678'

sig = sha1(key + message).hex()
sig

'a67d80beac0c46e6870c869881b519c7caaace56'

Since SHA1 is vulnerable to a length extension attack, based on the HMAC of a message and the lenght of the key a valid HMAC for an extended message can be created:

In [233]:
key_length = len(key)
message2 = b'An extension'


forged_message=sha1.pad(message, key_length) #  add the padding to the original message
l = len(forged_message)+key_length  # length (including key and padding) of the original message
h=tuple(int(sig[i*8:(i+1)*8],16) for i in range(5))  # status at the end of the original message

forged_message = forged_message + message2 # now we can add whatever we want

sha1(message2, h, l) == sha1(key + forged_message) # and still get a valid HAMC

True

Moroever, there is also [Keccak](https://keccak.team/). The class gives access to the individual operations. For example, here is the result of the permutation function $f$ applied to the initial state:

In [234]:
from kryptools import Keccak

keccak = Keccak(rate = 1600 - 2*256)
keccak.f()
keccak

f1258f7940e1dde7 84d5ccf933c0478a d598261ea65aa9ee bd1547306f80494d 8b284e056253d057
ff97a42d7f8e6fd4 90fee5a0a44647c4 8c5bda0cd6192e76 ad30a6f71b19059c 30935ab7d08ffc64
eb5aa93f2317d635 a9a6e6260d712103 81a57c16dbcf555f 43b831cd0347c826 01f22f1a11a5569f
05e5635a21d9ae61 64befef28cc970f2 613670957bc46611 b87c5a554fd00ecb 8c3ee88a1ccf32c8
940c7922ae3a2614 1841f924a2c509e4 16f53526e70465c2 75f644e97f30a13b eaf1ff7b5ceca249

You can `.reset()` the state and absorbe a message using the `.absorb()` method. Note, that messages are bufferd until a full block is available. Once you start squeezing, the remaining buffer gets padded and absorbed.

In [235]:
keccak.reset()
keccak.absorb(b'abc')
keccak.squeeze(10)

b'N\x03ez\xeaE\xa9O\xc7\xd4'

In addition, there is [SHA-3](https://en.wikipedia.org/wiki/SHA-3). To compute SHA-3-256 of a given string use:

In [236]:
from kryptools import SHA3

sha3 = SHA3(256) # SHA-3-256
sha3("abc").hex()

'3a985da74fe225b2045c172d6bd390bd855f086e3e9d525b46bfe24511431532'

Or you can use SHAKE to create pseudorandom bytes. To absorb a given string and extract a given number of bytes use:

In [237]:
from kryptools import SHAKE

shake = SHAKE(b'secret', n = 128)  # SHAKE-128
shake(8)

b'O:dn&\xb2i>'

You can extract furhter bytes by making further calls.